<a href="https://colab.research.google.com/github/bhuvnk/EVA/blob/master/Assignment17/Shufflenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf

import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()



## Adding the library folder to this notebook

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Setting the library folder to root path
import os
os.getcwd()
root_path = '/content/drive/My Drive/EVA2019/Session18 ShuffleNet/YALib' #change dir to your project folder

os.chdir(root_path)
os.getcwd()

'/content/drive/My Drive/EVA2019/Session18 ShuffleNet/YALib'

## CIfar10 download converted to TF rec

In [3]:
from generate_tfrecords import CIFAR_TFRecords

data_dir = '/content/'
CIFAR_TFRecords(data_dir)

Download from https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and extract.
Instructions for updating:
Please write your own downloading logic.
Generating /content/train.tfrecords


Generating /content/eval.tfrecords
Done!


In [0]:
filename_train = 'train.tfrecords' 
filename_test = 'eval.tfrecords' 
HEIGHT = 32
WIDTH = 32
DEPTH = 3

## Normalize and Augment And final Data Supplier

In [0]:
def normalize(image):
  """Convert `image` from [0, 255] -> [0, 1] floats."""
  image = tf.cast(image, tf.float32) * (1. / 255)
  return image  

def preprocess(image):
    """Preprocess a single image in [height, width, depth] layout."""
    # Pad 4 pixels on each dimension of feature map, done in mini-batch
    # image = tf.image.resize_image_with_crop_or_pad(image, 40, 40)
    image = tf.random_crop(image, [32, 32, 3])
    image = tf.image.random_flip_left_right(image)
    return image
def replace_slice(input_: tf.Tensor, replacement, begin) -> tf.Tensor:
    inp_shape = tf.shape(input_)
    size = tf.shape(replacement)
    padding = tf.stack([begin, inp_shape - (begin + size)], axis=1)
    replacement_pad = tf.pad(replacement, padding)
    mask = tf.pad(tf.ones_like(replacement, dtype=tf.bool), padding)
    return tf.where(mask, replacement_pad, input_)
def cutout(x: tf.Tensor, h: int=8, w: int=8, c: int = 3) -> tf.Tensor:
    shape = tf.shape(x)
    x0 = tf.random.uniform([], 0, shape[0] + 1 - h, dtype=tf.int32)
    y0 = tf.random.uniform([], 0, shape[1] + 1 - w, dtype=tf.int32)
    
    x = replace_slice(x, tf.random.uniform([h, w, c], dtype = tf.float32), [x0, y0, 0])
#     x = replace_slice(x, tf.zeros([h, w, c], dtype = tf.float32), [x0, y0, 0])
    return x  

In [0]:
def parser(serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
    # input format.
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })
    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape([DEPTH * HEIGHT * WIDTH])
    image = normalize(image)

    # Reshape from [depth * height * width] to [depth, height, width].
    image = tf.cast(
        tf.transpose(tf.reshape(image, [DEPTH, HEIGHT, WIDTH]), [1, 2, 0]),
        tf.float32)
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, 10)

    # Custom preprocessing.
    image = preprocess(image)
    image = cutout(image)

    return image, label  
  
def parser_test(serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
    # input format.
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.int64),
        })
    image = tf.decode_raw(features['image'], tf.uint8)
    image.set_shape([DEPTH * HEIGHT * WIDTH])
    image = normalize(image)

    # Reshape from [depth * height * width] to [depth, height, width].
    image = tf.cast(
        tf.transpose(tf.reshape(image, [DEPTH, HEIGHT, WIDTH]), [1, 2, 0]),
        tf.float32)
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label, 10)

    return image, label    

# Writing Shuffle Net

Had a lot of help from better coders from the TSAI

In [0]:
def _stage(tensor, nb_groups, in_channels, out_channels, repeat):
  x = _shufflenet_unit(tensor, nb_groups, in_channels, out_channels, 2)
  for _ in range(repeat):
    x = _shufflenet_unit(x, nb_groups, out_channels, out_channels, 1)
  return x

# Pointwise Grouped convs
def _pw_group(tensor, nb_groups, in_channels, out_channels):
  """Pointwise grouped convolution."""
  nb_chan_per_grp = in_channels // nb_groups
  pw_convs = []
  for grp in range(nb_groups):
    x = tf.keras.layers.Lambda(lambda x: x[:, :, :, nb_chan_per_grp * grp: nb_chan_per_grp * (grp + 1)])(tensor)
    grp_out_chan = int(out_channels / nb_groups + 0.5)
    pw_convs.append(
        tf.keras.layers.Conv2D(grp_out_chan,
                               kernel_size=(1, 1),
                               padding='same',
                               use_bias=False,
                               strides=1)(x)
                    )
  return tf.keras.layers.Concatenate(axis=-1)(pw_convs)



def _shuffle(x, nb_groups):
  def shuffle_layer(x):
    _, w, h, n = tf.keras.backend.int_shape(x)
    nb_chan_per_grp = n // nb_groups
    
    x = tf.reshape(x, (-1, w, h, nb_chan_per_grp, nb_groups))
    x = tf.keras.backend.permute_dimensions(x, (0, 1, 2, 4, 3)) # Transpose only grps and chs
    x = tf.reshape(x, (-1, w, h, n))
    return x
    
  return tf.keras.layers.Lambda(shuffle_layer)(x)

def _shufflenet_unit(tensor, nb_groups, in_channels, out_channels, strides, shuffle=True, bottleneck=8):
  bottleneck_channels = out_channels // bottleneck
  x = _pw_group(tensor, nb_groups, in_channels, bottleneck_channels)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  
  if shuffle:
    x = _shuffle(x, nb_groups)
    
  x = tf.keras.layers.DepthwiseConv2D(kernel_size=(3, 3),
                                      padding='same',
                                      use_bias=False,
                                      strides=strides)(x)                                  
  x = tf.keras.layers.BatchNormalization()(x)
  
  x = _pw_group(x, nb_groups, bottleneck_channels,
                out_channels if strides < 2 else out_channels - in_channels)
  x = tf.keras.layers.BatchNormalization()(x)
  
  
  if strides < 2:
    x = tf.keras.layers.Add()([tensor, x])
  else:
    avg = tf.keras.layers.AveragePooling2D(pool_size=(3, 3),
                                           strides=2,
                                           padding='same')(tensor)
    x = tf.keras.layers.Concatenate(axis=-1)([avg, x])

  x = tf.keras.layers.Activation('relu')(x)
  return x

def _info(nb_groups):
  return {
      1: [24, 144, 288, 576],
      2: [24, 200, 400, 800],
      3: [24, 240, 480, 960],
      4: [24, 272, 544, 1088],
      8: [24, 384, 768, 1536]
      }[nb_groups], [None, 3, 7, 3]


## Putting it all together

def ShuffleNet(input_shape, nb_classes, include_top=True, weights=None, nb_groups=3):
  x_in = tf.keras.Input(shape=input_shape)
  x = tf.keras.layers.Conv2D(24,
               kernel_size=(3, 3),
               strides=1,
               use_bias=False,
               padding='same')(x_in)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Activation('relu')(x)
  
  # x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3),
  #                                  strides=2,
  #                                  padding='same')(x)
  
  channels_list, repeat_list = _info(nb_groups)
  for i, (out_channels, repeat) in enumerate(zip(channels_list[1:], repeat_list[1:]), start=1):
    x = _stage(x, nb_groups, channels_list[i-1], out_channels, repeat)
    
  if include_top:
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(nb_classes, activation='softmax')(x)
  
  model = tf.keras.Model(inputs=x_in, outputs=x)
  
  if weights is not None:
    model.load_weights(weights, by_name=True)
  
  return model


In [8]:
model = ShuffleNet((32,32,3),10)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 24)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 24)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

## Training the shuffle net

In [0]:
# Hyperparameters
num_classes = 10
dropout_rate = 0.1
batch_size = 512
num_epochs = 50

In [10]:
# TEST TFRecords
dataset_test = tf.data.TFRecordDataset(filename_test).repeat(1)

dataset_test = dataset_test.map(parser_test, num_parallel_calls=batch_size)
dataset_test = dataset_test.shuffle(buffer_size=10000) 

dataset_test = dataset_test.batch(batch_size)

In [0]:
import time
sgd = tf.keras.optimizers.SGD(lr=0.005, decay=4e-5, momentum=0.9, nesterov=True)

# tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
                optimizer=sgd,
                metrics=['accuracy'])
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
start_time = time.time()
for zz in range(1,num_epochs+1):
  dataset_train = tf.data.TFRecordDataset(filename_train)#.repeat(1)
  dataset_train = dataset_train.apply(tf.data.experimental.shuffle_and_repeat(50000,1))
  dataset_train = dataset_train.apply(tf.data.experimental.map_and_batch(parser,batch_size, num_parallel_calls=batch_size))

  print("TrainEpochs number{}/{}".format(zz,num_epochs))
  
  model.fit_generator(dataset_train,steps_per_epoch=int(50000/batch_size),epochs=1,verbose=1)#, callbacks = [reduce_lr_loss])#,validation_data=dataset_test)

  print("Test Epochs number{}/{}".format(zz,num_epochs))
  model.evaluate_generator(dataset_test,steps=int(10000/batch_size),verbose=1)
  print("  ")
  print("------------------------",time.time()-start_time)

Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
TrainEpochs number1/50
97/97 [==============================] - 58s 602ms/step - loss: 3.3950 - acc: 0.2405
Test Epochs number1/50
19/19 [==============================] - 3s 181ms/step - loss: 2.6499 - acc: 0.1002
  
------------------------ 62.34569048881531
TrainEpochs number2/50
97/97 [==============================] - 56s 579ms/step - loss: 3.0841 - acc: 0.3484
Test Epochs number2/50
19/19 [==============================] - 3s 182ms/step - loss: 3.3486 - acc: 0.1002
  

## Misclassified images

In [0]:
from keras.datasets import cifar10
(train_features, train_labels), (test_features, test_labels) = cifar10.load_data()

In [0]:
import numpy as np
import matplotlib.pyplot as plt

j,t = 0, 0
X_test_wrong = []
X_test_wrong_raw = []
for i in range(len(test_features)):
  img_data = test_features[i].astype('float32')
  img_data /= 255
  img_data = np.expand_dims(img_data, axis=0)
#   img_data = test_features[i].reshape(1, 32, 32,3)
  t += 1
#   print(model.predict(img_data)[0])
#   print(np.argmax(model.predict(img_data)[0]))
#   print(test_labels[i][0])
  if np.argmax(model.predict(img_data)[0]) != test_labels[i][0]:
    X_test_wrong.append((img_data, np.argmax(model.predict(img_data)[0]), test_labels[i][0]))
    X_test_wrong_raw.append((test_features[i], np.argmax(model.predict(img_data)[0]), test_labels[i][0]))
    j += 1
  if j >= 50:
    break
print('totla false preds {} for samples {}'.format(j,t))

In [0]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

plt.figure(figsize=(35,35))
batch_size = 25
i = 0
for img in X_test_wrong:
  ax = plt.subplot(10,5,i+1)
  ax.set_title("pred: {}   actu: {}".format(class_names[img[1]], class_names[img[2]]), color='r')
  plt.imshow(np.reshape(np.uint8(255 * img[0]), (32,32,-1)), cmap=plt.cm.gray)
  i += 1